In [3]:
%matplotlib inline

In [5]:
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import

In [15]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
flags = tf.app.flags
FLAGS = flags.FLAGS 

tf.logging.set_verbosity(tf.logging.INFO)
# our application logic wil be added here
if __name__ == "main":
    tf.app.run()   

In [25]:
def cnn_model_fn(features, labels, mode):
    # input layer 
    input_layer = tf.reshape(features["X"], [-1, 28, 28, 1])
    
    # convoluation layer 1
    conv1 = tf.layers.conv2d(inputs = input_layer,
                            filters = 32,
                            kernel_size = [5,5],
                            padding = "same",
                            activation = tf.nn.relu)
    # Pooling layers 1
    pool1 = tf.layers.max_pooling2d(inputs= conv1, pool_size= [2,2], strides= 2)
    
    # convoluation layers 2
    conv2 = tf.layers.conv2d(inputs = pool1, 
                            filters = 64,
                            kernel_size = [5,5],
                            padding = "same",
                            activation = tf.nn.relu)
    ## pooling layer 2
    pool2 = tf.layers.max_pooling2d(inputs= conv2, pool_size= [2,2], strides= 2)
    
    # adding Dense layer
    pool2_flat = tf.reshape(pool2, [-1, 7*7*64])
    dense = tf.layers.dense(inputs = pool2_flat, units = 1024, activation= tf.nn.relu)
    dropout = tf.layer.dropout(inputs = dense, rate = 0.4, training=mode == tf.estimator.ModeKeys.Train)
    
    #logit layer
    logits = tf.layers.dense(inputs = dropout, units = 10)
    
    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }
    if mode == tf.estimator.ModeKeys.PREDICT:
            return tf.estimator.EstimatorSpec(mode = mode, predictions = predictions)
        # calculate loss (for both train and eval modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels = labels, logits = logits)
        
        # configure the training op (for Train mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
            optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.001)
            train_op = optimizer.minimize(loss = loss, global_step = tf.get_global_step())
    return tf.estimator.EstimatorSpec(mode = mode, loss = loss, train_op = train_op)


    ## Addding the accuracy evaluation metrics (For EVAL mode)
    eval_metrics_ops = {
        "accuracy": tf.metrics.accuracy(
        labels = labels, predictions = predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode = mode, loss = loss, eval_metrics_ops = eval_metrics_ops)

In [49]:
## training and evaluating the mnist classifier 
## load the train and test data 

def main(unused_arg):
    # load the training and eval data 
    mnist = tf.contrib.learn.datasets.load_dataset("mnist")
    train_data = mnist.train.images # Return np.array
    train_labels = np.array(mnist.train.labels, dtype = np.int32)
    eval_data = mnist.test.images
    eval_labels = np.array(mnist.train.labels, dtype = np.int32)
    
    # create the estimator 
    mnist_classifier = tf.estimator.Estimator(
    model_fn = cnn_model_fn, model_dir = "/tmp/mnist_convnet_model")

    # setup the  logging for predictions 
    tensors_to_log = {"probabilities": "softmax_tensorflow"}
    logging_hook = tf.train.LoggingTensorHook(
    tensors = tensors_to_log, every_n_iter = 50)

    ## training the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": train_data},
        y= train_labels,
        batch_size=100,
        num_epochs=None,
        shuffle=True)
    mnist_classifier.train(
        input_fn=train_input_fn,
        steps=20000,
        hooks=[logging_hook])
    ## Evaluate the Model and print results 
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x = {"x": eval_data},
        y = eval_labels,
        num_epochs = 1, shuffle = False)
    eval_results = mnist_classifier.evaluate(input_fn = eval_input_fn)
    print(eval_results)